#Finetuning speecht5 modified for streaming mode (t5 normal)




Major variables

In [ ]:
dataset_name = "kazeric/DVoice-VoxLingua107"
dataset_config_name = "sw"
model_name = "kazeric/speecht5_sw_bible"
model_revision = "main"
dataset_text_column = "words"
dataset_audio_column = "file_path"
output_dir = "speecht5_sw_bible"
hub_id = "kazeric/speecht5_sw_bible"
max_steps=500
eval_steps=500
save_steps=500
revision="second_iter"
commit_message="interesting commit"


In [ ]:
!pip install transformers datasets soundfile speechbrain accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 787.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# create the branch to be used
from huggingface_hub import create_branch

if revision == "":
    revision = "main"
else:
  try:
    create_branch(repo_id=hub_id, branch=revision)
  except:
    print("Branch already exists")

Branch already exists


In [ ]:
from datasets import load_dataset, Audio, DatasetDict

dataset = DatasetDict()


dataset["train"] = load_dataset(dataset_name, split="train", streaming= True)
dataset["test"] = load_dataset(dataset_name, split="test", streaming= True)


README.md:   0%|          | 0.00/131 [00:00<?, ?B/s]

In [ ]:
dataset["train"] = dataset["train"].cast_column(dataset_audio_column, Audio(sampling_rate=16000))
dataset["test"]= dataset["test"].cast_column(dataset_audio_column, Audio(sampling_rate=16000))

In [ ]:
from transformers import SpeechT5Processor


processor = SpeechT5Processor.from_pretrained(model_name, revision=model_revision)

preprocessor_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [ ]:
tokenizer = processor.tokenizer

In [ ]:
# def extract_unique_characters(generator):
#     vocabulary = set()  # Using a set to store unique characters

#     for item in generator:

#         text = item[dataset_text_column]
#         # Add each unique character from the text to the vocabulary set
#         vocabulary.update(set(text))  # Adds each character to the set

#     return vocabulary

# dataset_vocab = extract_unique_characters(dataset["train"])


# tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

# dataset_vocab - tokenizer_vocab

In [ ]:
# replacements = [
#     ("à", "a"),
#     ("ç", "c"),
#     ("è", "e"),
#     ("ë", "e"),
#     ("í", "i"),
#     ("ï", "i"),
#     ("ö", "o"),
#     ("ü", "u"),
# ]


# def cleanup_text(inputs):
#     for src, dst in replacements:
#         inputs[dataset_text_column] = inputs[dataset_text_column].replace(src, dst)
#     return inputs


# dataset["train"] = dataset["train"].map(cleanup_text)
# dataset["test"] = dataset["test"].map(cleanup_text)

In [ ]:
# from collections import defaultdict

# speaker_counts = defaultdict(int)

# for example in dataset["train"]:
#     speaker_counts[example["speaker_id"]] += 1

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure()
# plt.hist(speaker_counts.values(), bins=20)
# plt.ylabel("Speakers")
# plt.xlabel("Examples")
# plt.show()

In [ ]:
# def select_speaker(speaker_id):
#     return 100 <= speaker_counts[speaker_id] <= 400


# dataset["train"] = dataset["train"].filter(select_speaker, input_columns=["speaker_id"])
# dataset["test"] = dataset["test"].filter(select_speaker, input_columns=["speaker_id"])



In [ ]:
# len(set(dataset["speaker_id"]))

In [ ]:
# len(dataset)

In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

<ipython-input-23-f28baad2ea60>:3: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
def prepare_dataset(example):
    audio = example[dataset_audio_column]

    example = processor(
        text=example[dataset_text_column],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [ ]:
# processed_example = prepare_dataset(dataset[0])
# list(processed_example.keys())

In [ ]:
# processed_example["speaker_embeddings"].shape

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure()
# plt.imshow(processed_example["labels"].T)
# plt.show()

In [ ]:
dataset["train"] = dataset["train"].map(prepare_dataset, remove_columns=dataset["train"].column_names)
dataset["test"] = dataset["test"].map(prepare_dataset, remove_columns=dataset["test"].column_names)

In [ ]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200


dataset["train"] = dataset["train"].filter(is_not_too_long, input_columns=["input_ids"])
dataset["test"] = dataset["test"].filter(is_not_too_long, input_columns=["input_ids"])


In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [ ]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(model_name, revision=model_revision)

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=max_steps,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=save_steps,
    eval_steps=eval_steps,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
    hub_model_id=hub_id,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

<ipython-input-35-3e9fd9c3f140>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

<ipython-input-30-2cc8fd78303e>:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["speaker_embeddings"] = torch.tensor(speaker_features)


Step,Training Loss,Validation Loss
500,0.677300,0.640398


TrainOutput(global_step=500, training_loss=0.8380959739685059, metrics={'train_runtime': 951.2568, 'train_samples_per_second': 8.41, 'train_steps_per_second': 0.526, 'total_flos': 1044658303219872.0, 'train_loss': 0.8380959739685059, 'epoch': 2.012})

In [ ]:
trainer.push_to_hub(commit_message=commit_message , revision=revision)

CommitInfo(commit_url='https://huggingface.co/kazeric/speecht5_sw_bible/commit/309d6f0ab6effa8f363d2f7372d359468896a638', commit_message='this one is a test to see how revisions will work', commit_description='', oid='309d6f0ab6effa8f363d2f7372d359468896a638', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kazeric/speecht5_sw_bible', endpoint='https://huggingface.co', repo_type='model', repo_id='kazeric/speecht5_sw_bible'), pr_revision=None, pr_num=None)

#Finetuning speecht5 using streaming mode with iter tools (t5 rar opt)

In [ ]:
dataset_name = "kazeric/OpenBible_Swahili_book_split"
dataset_revision = "resampled_whisper"
dataset_config_name = "sw"
model_name = "Lingua-connect/speecht5_sw_bible"
model_revision = "main"
dataset_text_column = "transcript"
dataset_audio_column = "audio"
output_dir = "speecht5_sw_bible"
hub_id = "Lingua-connect/speecht5_sw_bible"
max_steps=2500
eval_steps=500
save_steps=500
revision="Second_epoch"
commit_message="second epoch"

In [ ]:
!pip install datasets soundfile speechbrain accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers==4.49.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 92.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Helper cell to get the books

In [ ]:
# create the branch to be used
from huggingface_hub import create_branch

if revision == "":
    revision = "main"
else:
  try:
    create_branch(repo_id=hub_id, branch=revision)
  except:
    print("Branch already exists")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Branch already exists


In [ ]:
all_books = [
    "GEN", "EXO", "LEV", "NUM", "DEU", "JOS", "JDG", "RUT", "1SA", "2SA",
    "1KI", "2KI", "1CH", "2CH", "EZR", "NEH", "EST", "JOB", "PSA", "PRO",
    "ECC", "SNG", "ISA", "JER", "LAM", "EZK", "DAN", "HOS", "JOL", "AMO",
    "OBA", "JON", "MIC", "NAM", "HAB", "ZEP", "HAG", "ZEC", "MAL", "MAT",
    "MRK", "LUK", "JHN", "ACT", "ROM", "1CO", "2CO", "GAL", "EPH", "PHP",
    "COL", "1TH", "2TH", "1TI", "2TI", "TIT", "PHM", "HEB", "JAS", "1PE",
    "2PE", "1JN", "2JN", "3JN", "JUD", "REV"
]

#shuffle the list
# split the list by 80/20
from sklearn.model_selection import train_test_split

train_books, test_books = train_test_split(all_books, test_size=0.2, random_state=42)

print("Train Books:", train_books)
print("Test Books:", test_books)

Train Books: ['HAB', 'EZK', 'NUM', 'JOB', 'MRK', '1SA', 'JDG', 'JOL', 'DEU', 'EPH', 'HAG', 'PRO', 'PHM', '1PE', 'JAS', 'COL', 'JON', 'ROM', 'JUD', 'NEH', 'HOS', 'DAN', 'LAM', 'GAL', '2KI', 'MIC', '1JN', '2CO', 'ZEC', 'AMO', 'ACT', 'PHP', 'EXO', 'SNG', 'LEV', '1TI', 'MAT', 'ZEP', '2TH', 'JER', 'TIT', '1KI', 'ISA', 'PSA', 'HEB', 'MAL', 'ECC', '2PE', 'RUT', 'JHN', 'EZR', '1TH']
Test Books: ['2TI', '2JN', 'GEN', '1CO', 'JOS', '3JN', 'EST', '1CH', 'REV', 'OBA', 'NAM', '2SA', 'LUK', '2CH']


Helper to print examples

In [ ]:
def print_examples(dataset, num_examples=3):
    for i, example in enumerate(dataset):
        print("\n")
        print(example)
        if i >= num_examples:
            break

Load the dataset and merge them into one iteratable object

In [ ]:
from datasets import load_dataset
from itertools import chain, tee

repo_id = dataset_name

# Load all configurations in streaming mode
train_datasets = [load_dataset(repo_id, name=config+"_clean", split="train",revision=dataset_revision, streaming=True) for config in train_books]
test_datasets = [load_dataset(repo_id, name=config+"_clean", split="test",revision=dataset_revision , streaming=True) for config in test_books]

# Merge all streamed datasets into one iterable
merged_train_dataset = chain(*train_datasets)
merged_test_dataset = chain(*test_datasets)


# # Iterate through the merged dataset and print 5 samples
# print_examples(merged_train_dataset)
# print_examples(merged_test_dataset)

README.md:   0%|          | 0.00/32.3k [00:00<?, ?B/s]

In [ ]:
# from datasets import load_dataset, Audio

# dataset = load_dataset("facebook/voxpopuli", "nl", split="train")
# len(dataset)

Resample the audio

In [ ]:
import librosa

def resample_audio_example(example, target_sr=16000):
    if dataset_audio_column in example and example[dataset_audio_column] is not None:
        audio_array = example[dataset_audio_column]["array"]
        original_sr = example[dataset_audio_column]["sampling_rate"]

        # Resample only if the sample rate is different
        if original_sr != target_sr:
            example[dataset_audio_column]["array"] = librosa.resample(audio_array, orig_sr=original_sr, target_sr=target_sr)
            example[dataset_audio_column]["sampling_rate"] = target_sr  # Update the sample rate

    return example

In [ ]:
resampled_train_dataset = (resample_audio_example(example, 16000) for example in merged_train_dataset)
resampled_test_dataset = (resample_audio_example(example, 16000) for example in merged_test_dataset)

# # Print 5 examples to verify
# print_examples(resampled_train_dataset)
# print_examples(resampled_test_dataset)

In [ ]:
# dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

Import the speech t5 processor

In [ ]:
from transformers import SpeechT5Processor

processor = SpeechT5Processor.from_pretrained(model_name)

preprocessor_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

load the tokenizer

In [ ]:
tokenizer = processor.tokenizer

Extract all unique characters

If i accidentaly stop the function mid run it returns an empty set in subsequent runs, why ?

This part how ever is for the transcription of the swahili dataset as it is already normalised

In [ ]:
# def extract_unique_characters(generator):
#     vocabulary = set()  # Using a set to store unique characters

#     for item in generator:
#         # Assuming each dictionary contains a key 'text' for the text content
#         text = item["transcript"]
#         # Add each unique character from the text to the vocabulary set
#         vocabulary.update(set(text))  # Adds each character to the set

#     return vocabulary

# dataset_vocab = extract_unique_characters(resampled_train_dataset)


# tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

# dataset_vocab - tokenizer_vocab


In [ ]:
# replacements = [
#     ("à", "a"),
#     ("ç", "c"),
#     ("è", "e"),
#     ("ë", "e"),
#     ("í", "i"),
#     ("ï", "i"),
#     ("ö", "o"),
#     ("ü", "u"),
# ]


# def cleanup_text(inputs):
#     for src, dst in replacements:
#         inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
#     return inputs


# dataset = dataset.map(cleanup_text)

balancing speakers

In [ ]:
# from collections import defaultdict

# speaker_counts = defaultdict(int)

# for speaker_id in dataset["speaker_id"]:
#     speaker_counts[speaker_id] += 1

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure()
# plt.hist(speaker_counts.values())
# plt.ylabel("Speakers")
# plt.xlabel("Examples")
# plt.show()

In [ ]:
# def select_speaker(speaker_id):
#     return 100 <= speaker_counts[speaker_id] <= 400


# dataset = dataset.filter(select_speaker, input_columns=["speaker_id"])

In [ ]:
# len(set(dataset["speaker_id"]))

In [ ]:
# len(dataset)

In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover
<ipython-input-23-f28baad2ea60>:3: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/hyperparams.yaml' -> '/tmp/speechbrain/spkrec-xvect-voxceleb/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _load
DEBUG:speechbrain.utils.checkpoints:Registered parameter transfer hook for _load
/usr/local/lib/python3.11/dist-packages/speechbrain/utils/autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook fo

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/embedding_model.ckpt' -> '/tmp/speechbrain/spkrec-xvect-voxceleb/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /tmp/speechbrain/spkrec-xvect-voxceleb/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


...

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/mean_var_norm_emb.ckpt' -> '/tmp/speechbrain/spkrec-xvect-voxceleb/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /tmp/speechbrain/spkrec-xvect-voxceleb/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/classifier.ckpt' -> '/tmp/speechbrain/spkrec-xvect-voxceleb/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /tmp/speechbrain/spkrec-xvect-voxceleb/classifier.ckpt
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/label_encoder.txt' -> '/tmp/speechbrain/spkrec-xvect-voxceleb/label_encoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["label_encoder"] = /tmp/speechbrain/spkrec-xvect-voxceleb/label_encoder.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): embedding_model -> /tmp/speechbrain/spkrec-xvect-voxceleb/embedding_model.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): mean_var_norm_emb -> /tmp/speechbrain/spkrec-xvect-voxceleb/mean_var_norm_emb.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): classifier -> /tmp/speechbra

In [ ]:
def prepare_dataset(example):
    audio = example[dataset_audio_column]

    example = processor(
        text=example[dataset_text_column],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

show how speech t5 handles speech data

In [ ]:
# processed_example = prepare_dataset(dataset[0])
# list(processed_example.keys())

In [ ]:
# processed_example["speaker_embeddings"].shape

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure()
# plt.imshow(processed_example["labels"].T)
# plt.show()

In [ ]:
prepared_train_dataset = (prepare_dataset(example) for example in resampled_train_dataset)
prepared_test_dataset = (prepare_dataset(example) for example in resampled_test_dataset)

# print_examples(prepared_train_dataset)
# print_examples(prepared_test_dataset)

In [ ]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200


filtered_train_dataset = (example for example in prepared_train_dataset if is_not_too_long(example["input_ids"]))
filtered_test_dataset = (example for example in prepared_test_dataset if is_not_too_long(example["input_ids"]))

# print_examples(filtered_train_dataset)
# print_examples(filtered_test_dataset)

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [ ]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(model_name, revision=model_revision)

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

Find the length of the data set

In [ ]:
# trainlength = sum(1 for _ in filtered_train_dataset)
# testlength = sum(1 for _ in filtered_test_dataset)
# print(trainlength)
# print(testlength)

trainlength = 20766
testlength = 5754

In [ ]:
import torch
from torch.utils.data import IterableDataset



class CustomIterableDataset(IterableDataset):
    def __init__(self, generator_func, length):
        self.generator_func = generator_func
        self.length = length

    def __iter__(self):
        yield from self.generator_func()  # Iterate through the generator

    def __len__(self):
        return self.length  # Return the desired length

# Create an instance of the custom dataset
train_dataset = CustomIterableDataset(lambda: filtered_train_dataset, length=trainlength)
eval_dataset = CustomIterableDataset(lambda: filtered_test_dataset, length=testlength)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=-1,
    num_train_epochs=1,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=2,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
    hub_model_id=hub_id,
)



In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=processor,
)

<ipython-input-40-7997c04297b0>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

<ipython-input-30-2cc8fd78303e>:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["speaker_embeddings"] = torch.tensor(speaker_features)


Epoch,Training Loss,Validation Loss
1,0.467000,0.444556


TrainOutput(global_step=1298, training_loss=0.4957778215040962, metrics={'train_runtime': 3513.5591, 'train_samples_per_second': 5.91, 'train_steps_per_second': 0.369, 'total_flos': 3069430122192192.0, 'train_loss': 0.4957778215040962, 'epoch': 1.0})

In [ ]:
# not to be confused with the configs at the top these ones are for the new model once complete
kwargs = {
    "dataset_tags": "bookbot/OpenBible_Swahili",
    "dataset": "OpenBible_Swahili_book_split",  # a 'pretty' name for the training dataset
    "language": "sw",
    "model_name": "SpeechT5_Small_swahili_bible",  # a 'pretty' name for your model
    "finetuned_from": "microsoft/speecht5_tts",
    "tasks": "text-to-speech",
}

In [ ]:
trainer.push_to_hub(commit_message=commit_message , revision=revision)

KeyboardInterrupt: 

#Automatic speech recognition - modified to streaming mode with iter rools modified (asr rar opt)

##How to fine-tune an ASR system with the traininer API

I might not spend time here as this is the back up plan, but if speech t5 doesnt work out then we can use the whisper model to

- train ASR in swahili then giriama to utilise transfere learning
- train ASR in giriama alone to check it viability as well

In [ ]:
dataset_name = "kazeric/OpenBible_Swahili_book_split"
dataset_revision = "resampled_whisper"
dataset_config_name = "sw"
model_name = "Lingua-connect/whisper-small-sw-bible"
model_revision="main"
dataset_text_column = "transcript"
dataset_audio_column = "audio"
output_dir = "whisper-small-sw-bible"
hub_id = "Lingua-connect/whisper-small-sw-bible"
task="transcribe"
language = "swahili"
max_steps=2500
eval_steps=500
save_steps=500
revision="Second_epoch"
commit_message="interesting commit"

In [ ]:
!pip install datasets

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# create the branch to be used
from huggingface_hub import create_branch

if revision == "":
    revision = "main"
else:
  try:
    create_branch(repo_id=hub_id, branch=revision)
  except:
    print("Branch already exists")

Helper to get the books

In [ ]:
all_books = [
    "GEN", "EXO", "LEV", "NUM", "DEU", "JOS", "JDG", "RUT", "1SA", "2SA",
    "1KI", "2KI", "1CH", "2CH", "EZR", "NEH", "EST", "JOB", "PSA", "PRO",
    "ECC", "SNG", "ISA", "JER", "LAM", "EZK", "DAN", "HOS", "JOL", "AMO",
    "OBA", "JON", "MIC", "NAM", "HAB", "ZEP", "HAG", "ZEC", "MAL", "MAT",
    "MRK", "LUK", "JHN", "ACT", "ROM", "1CO", "2CO", "GAL", "EPH", "PHP",
    "COL", "1TH", "2TH", "1TI", "2TI", "TIT", "PHM", "HEB", "JAS", "1PE",
    "2PE", "1JN", "2JN", "3JN", "JUD", "REV"
]

#shuffle the list
# split the list by 80/20
from sklearn.model_selection import train_test_split

train_books, test_books = train_test_split(all_books, test_size=0.2, random_state=42)

print("Train Books:", train_books)
print("Test Books:", test_books)

Train Books: ['HAB', 'EZK', 'NUM', 'JOB', 'MRK', '1SA', 'JDG', 'JOL', 'DEU', 'EPH', 'HAG', 'PRO', 'PHM', '1PE', 'JAS', 'COL', 'JON', 'ROM', 'JUD', 'NEH', 'HOS', 'DAN', 'LAM', 'GAL', '2KI', 'MIC', '1JN', '2CO', 'ZEC', 'AMO', 'ACT', 'PHP', 'EXO', 'SNG', 'LEV', '1TI', 'MAT', 'ZEP', '2TH', 'JER', 'TIT', '1KI', 'ISA', 'PSA', 'HEB', 'MAL', 'ECC', '2PE', 'RUT', 'JHN', 'EZR', '1TH']
Test Books: ['2TI', '2JN', 'GEN', '1CO', 'JOS', '3JN', 'EST', '1CH', 'REV', 'OBA', 'NAM', '2SA', 'LUK', '2CH']


Helper to print examples

In [ ]:
def print_examples(dataset, num_examples=3):
    for i, example in enumerate(dataset):
        print("\n")
        print(example)
        if i >= num_examples:
            break

**Helper** to create a new iterator so that we can start from scratch

In [ ]:
def get_new_iterator(iterable):
    return iter(iterable)

Load the dataset and merge them into one iteratable object

> Add blockquote



In [ ]:
from datasets import load_dataset
from itertools import chain, tee

repo_id = dataset_name
revision = dataset_revision

# # Load all configurations in streaming mode
# train_datasets = [load_dataset(repo_id, name=config+"_clean", split="train",revision=revision, streaming=True) for config in train_books]
# test_datasets = [load_dataset(repo_id, name=config+"_clean", split="test",revision=revision , streaming=True) for config in test_books]

# # Merge all streamed datasets into one iterable
# merged_train_dataset = chain(*train_datasets)
# merged_test_dataset = chain(*test_datasets)

# #split the iterators in the this case so that we should always use 2 to print while modifying the main one will be 1
# merged_train_dataset1, merged_train_dataset2 = tee(merged_train_dataset)
# merged_test_dataset1, merged_test_dataset2 = tee(merged_test_dataset)

# print_examples(merged_train_dataset2, 1)
# print_examples(merged_train_dataset2, 1)


filter the unwanted columns

In [ ]:

# # Remove specific columns using a generator expression
# filtered_train_dataset = ({k: v for k, v in example.items() if k  in ["audio", "verse_text"]} for example in merged_train_dataset)

# filtered_test_dataset = ({k: v for k, v in example.items() if k  in ["audio", "verse_text"]} for example in merged_test_dataset)

# # filtered_train_dataset1, filtered_train_dataset2 = tee(filtered_train_dataset)
# # filtered_test_dataset1, filtered_test_dataset2 = tee(filtered_test_dataset)
# # Print 5 examples to verify

# print_examples(filtered_train_dataset2)
# print_examples(filtered_test_dataset2)

resample the audio

In [ ]:
import librosa


In [ ]:
def resample_audio_example(example, target_sr=16000):
    if dataset_audio_column in example and example[dataset_audio_column] is not None:
        audio_array = example[dataset_audio_column]["array"]
        original_sr = example[dataset_audio_column]["sampling_rate"]

        # Resample only if the sample rate is different
        if original_sr != target_sr:
            example[dataset_audio_column]["array"] = librosa.resample(audio_array, orig_sr=original_sr, target_sr=target_sr)
            example[dataset_audio_column]["sampling_rate"] = target_sr  # Update the sample rate

    return example


In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    model_name, language=language, task=task
)
sampling_rate = processor.feature_extractor.sampling_rate


In [ ]:



# resampled_train_dataset = (resample_audio_example(example, sampling_rate) for example in filtered_train_dataset)
# resampled_test_dataset = (resample_audio_example(example, sampling_rate) for example in filtered_test_dataset)

# # resampled_train_dataset1, resampled_train_dataset2 = tee(resampled_train_dataset)
# # resampled_test_dataset1, resampled_test_dataset2 = tee(resampled_test_dataset)
# # # Print 5 examples to verify
# # print_examples(resampled_train_dataset2)
# # print_examples(resampled_test_dataset2)

In [ ]:
def prepare_dataset(example):
    audio = example[dataset_audio_column]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example[dataset_text_column],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

[link text](https://)why are we removing the column names

In [ ]:
# prepared_train_dataset = (prepare_dataset(example) for example in resampled_train_dataset)
# prepared_test_dataset = (prepare_dataset(example) for example in resampled_test_dataset)

# # prepared_train_dataset1, prepared_train_dataset2 = tee(prepared_train_dataset)
# # prepared_test_dataset1, prepared_test_dataset2 = tee(prepared_test_dataset)

# # # Print 5 examples to verify
# # print_examples(prepared_train_dataset2)
# # print_examples(prepared_test_dataset2)


In [ ]:
def is_audio_in_length_range(length, max_input_length = 30.0):
    return length < max_input_length

In [ ]:
# filtered_train_dataset = (example for example in prepared_train_dataset if is_audio_in_length_range(example["input_length"]))
# filtered_test_dataset = (example for example in prepared_test_dataset if is_audio_in_length_range(example["input_length"]))

# filtered_train_dataset1, filtered_train_dataset2 = tee(filtered_train_dataset)
# filtered_test_dataset1, filtered_test_dataset2 = tee(filtered_test_dataset)

# print_examples(filtered_train_dataset2)
# print_examples(filtered_test_dataset2)


In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
!pip install evaluate

In [ ]:
!pip install jiwer

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


import evaluate

metric = evaluate.load("wer")

from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]

    # filtering step to only evaluate the samples that correspond to non-zero references:
    # Filter out empty strings after normalization for both predictions and references
    pred_str_norm = [p for p, l in zip(pred_str_norm, label_str_norm) if l and isinstance(l, str) and l.strip()]  # Check for empty string after strip
    label_str_norm = [l for l in label_str_norm if l and isinstance(l, str) and l.strip()] # Check for empty string after strip

    # Check if label_str_norm is empty and skip WER calculation if so
    if not label_str_norm:
        print("Warning: label_str_norm is empty, skipping WER calculation.")
        return {"wer_ortho": wer_ortho, "wer": float('nan')}  # or any suitable default value

    #**Remove the split to keep the strings as sentences, not words**
    # pred_str_norm = [p.split() if isinstance(p, str) else p for p in pred_str_norm]
    # label_str_norm = [l.split() if isinstance(l, str) else l for l in label_str_norm]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name, revision=model_revision)

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language=language, task=task, use_cache=True
    )

what exactly are steps and what are they used for?

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # name on the HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=max_steps,# increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    eval_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=save_steps,
    eval_steps=save_steps,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    hub_model_id=hub_id,
)



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Finding the lenght of the data set

In [ ]:
import torch
from torch.utils.data import IterableDataset



class CustomIterableDataset(IterableDataset):
    def __init__(self, generator_func):
        self.generator_func = generator_func

    def __iter__(self):
        yield from self.generator_func()  # Iterate through the generator



# Create an instance of the custom dataset
# Define functions to create datasets (call these whenever you need fresh datasets)
def create_train_dataset():
    # Recreate the iterators
    train_datasets = [load_dataset(repo_id, name=config+"_clean", split="train",revision=revision, streaming=True) for config in train_books]
    merged_train_dataset = chain(*train_datasets)
    filtered_train_dataset = ({k: v for k, v in example.items() if k  in [dataset_audio_column, dataset_text_column]} for example in merged_train_dataset)
    resampled_train_dataset = (resample_audio_example(example, sampling_rate) for example in filtered_train_dataset)
    prepared_train_dataset = (prepare_dataset(example) for example in resampled_train_dataset)
    final_train_dataset = (example for example in prepared_train_dataset if is_audio_in_length_range(example["input_length"]))
    return CustomIterableDataset(lambda: final_train_dataset)

def create_eval_dataset():
    # Recreate the iterators
    test_datasets = [load_dataset(repo_id, name=config+"_clean", split="test",revision=revision , streaming=True) for config in test_books]
    merged_test_dataset = chain(*test_datasets)
    filtered_test_dataset = ({k: v for k, v in example.items() if k  in [dataset_audio_column, dataset_text_column]} for example in merged_test_dataset)
    resampled_test_dataset = (resample_audio_example(example, sampling_rate) for example in filtered_test_dataset)
    prepared_test_dataset = (prepare_dataset(example) for example in resampled_test_dataset)
    final_test_dataset = (example for example in prepared_test_dataset if is_audio_in_length_range(example["input_length"]))
    return CustomIterableDataset(lambda: final_test_dataset)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=create_train_dataset(),
    eval_dataset=create_eval_dataset(),
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

<ipython-input-207-84e2fb8db511>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


how and where is the number of epochs specified in the model?

what is this error and can it be solved for to bring better results

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

In [ ]:
trainer.train()

KeyError: 'transcript'

In [ ]:
kwargs = {
    "dataset_tags": "bookbot/OpenBible_Swahili",
    "dataset": "OpenBible_Swahili_book_split",  # a 'pretty' name for the training dataset
    "language": "sw",
    "model_name": "Whisper_Small_swahili_bible",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(commit_message=commit_message , revision=revision)

#Automatic speech recognition - modified to streaming mode.(asr normal)


##How to fine-tune an ASR system with the traininer API

In [ ]:
dataset_name = "kazeric/DVoice-VoxLingua107"
dataset_config_name = "sw"
model_name = "openai/whisper-small"
dataset_text_column = "words"
dataset_audio_column = "file_path"
output_dir = "whisper-small-sw-normal"
hub_id = "Lingua-connect/whisper-small-sw-normal"
task="transcribe"
language = "swahili"
max_steps=2000
eval_steps=500
save_steps=500
revision=""
commit_message="interesting commit"

In [ ]:
# !pip install --upgrade torch torchvision torchaudio

I might not spend time here as this is the back up plan, but if speech t5 doesnt work out then we can use the whisper model to

- train ASR in swahili then giriama to utilise transfere learning
- train ASR in giriama alone to check it viability as well

In [ ]:
!pip install datasets

ERROR: Operation cancelled by user


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# create the branch to be used
from huggingface_hub import create_branch

if revision == "":
    revision = "main"
else:
  try:
    create_branch(repo_id=hub_id, branch=revision)
  except:
    print("Branch already exists")

In [ ]:
from datasets import load_dataset, DatasetDict

dataset = DatasetDict()

dataset["train"] = load_dataset(
    dataset_name, split="train", streaming=True
)
dataset["test"] = load_dataset(
    dataset_name, split="test", streaming=True
)

print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
dataset["train"] = dataset["train"].remove_columns([cols for cols in dataset["train"].column_names if cols not in [dataset_audio_column, dataset_text_column]])
dataset["test"] = dataset["test"].remove_columns([cols for cols in dataset["test"].column_names if cols not in [dataset_audio_column, dataset_text_column]])

print(dataset["train"])


In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    model_name, language=language, task=task
)

In [ ]:
next(iter(dataset["train"]))

In [ ]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate


dataset["train"] = dataset["train"].cast_column(dataset_audio_column, Audio(sampling_rate=sampling_rate))
dataset["test"] = dataset["test"].cast_column(dataset_audio_column, Audio(sampling_rate=sampling_rate))

In [ ]:
next(iter(dataset["train"]))

In [ ]:
def prepare_dataset(example):
    audio = example[dataset_audio_column]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example[dataset_text_column],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

[link text](https://)why are we removing the column names

In [ ]:
dataset["train"] = dataset["train"].map(
    prepare_dataset
)
dataset["test"] = dataset["test"].map(
    prepare_dataset
)

In [ ]:
max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

In [ ]:
dataset["train"] = dataset["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

In [ ]:
dataset["train"]

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
!pip install evaluate

In [ ]:
!pip install jiwer

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model_name, language=language, task=task, use_cache=True
)

what exactly are steps and what are they used for?

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=max_steps,
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    eval_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=save_steps,
    eval_steps=eval_steps,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    hub_model_id=hub_id,
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

how and where is the number of epochs specified in the model?

what is this error and can it be solved for to bring better results

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

In [ ]:
trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_13_0",
    "dataset": "Common Voice 13",  # a 'pretty' name for the training dataset
    "language": "sw",
    "model_name": "Whisper_Small_swahili_normal",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs,commit_message=commit_message , revision=revision)

#Testing the model


##ASR

In [11]:
import torch
print(torch.__version__)

2.6.0+cu124


In [12]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [13]:
#put all parameters here
model_name ="Lingua-Connect/whisper-small-sw-bible"
model_revision = "main"

In [14]:
!pip install transformers==4.49.0

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import pipeline

model_id = model_name  # update with your model id
pipe = pipeline("automatic-speech-recognition", model=model_id, revision=model_revision)

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

In [ ]:
import transformers

print(transformers.__version__)


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline


def transcribe_speech(filepath):
    output = pipe(
        filepath,
        max_new_tokens=256,
        generate_kwargs={
            "task": "transcribe",
            "language": "swahili",
        },  # update with the language you've fine-tuned on
        chunk_length_s=30,
        batch_size=8,
    )
    return output["text"]



In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

demo = gr.Blocks()

mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs=gr.components.Textbox(),
)

file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="upload", type="filepath"),
    outputs=gr.components.Textbox(),
)

with demo:
    gr.TabbedInterface(
        [mic_transcribe, file_transcribe],
        ["Transcribe Microphone", "Transcribe Audio File"],
    )

demo.launch(debug=True)

##TTS

In [ ]:
model_name =  "Lingua-Connect/speecht5_sw_bible"
model_revision = "giriama_bible_unscrubbed"
dataset_name = "kazeric/OpenBible_Swahili_book_split"
dataset_config_name = "HAB_clean"
dataset_text_column = "words"
dataset_audio_column = "audio"
text = "mbolea ya kuku hudza na vidudu here huke wa kuku"


In [ ]:
!pip install datasets speechbrain

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import SpeechT5Processor
isinstance

processor = SpeechT5Processor.from_pretrained(model_name)

In [ ]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(
   model_name,
)

In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/tmp/speechbrain/spkrec-xvect-voxceleb/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in /tmp/speechbrain/spkrec-xvect-voxceleb.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Using symlink found at '/tmp/speechbrain/spkrec-xvect-voxceleb/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /tmp/speechbrain/spkrec-xvect-voxceleb/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Using symlink found at '/tmp/speechbrain/spkrec-xvect-voxceleb/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /tmp/speechbrain/spkrec-xvect-voxceleb/mean_var_norm_emb.ck

In [ ]:
from datasets import load_dataset
import librosa

dataset = load_dataset(dataset_name, dataset_config_name, split="train", streaming = True)

example = next(iter(dataset))

print(example)

#get the info from the data item
original_audio =example[dataset_audio_column]['array']
original_sr = example[dataset_audio_column]['sampling_rate']
target_sr = 16000

#resample the audio to 16K as required by the model
example[dataset_audio_column]['array'] = librosa.resample(original_audio, orig_sr=original_sr, target_sr=target_sr)
example[dataset_audio_column]['sampling_rate'] = target_sr
print(example)

embeddings = create_speaker_embedding(example[dataset_audio_column]["array"])

embeddings = torch.tensor(embeddings).unsqueeze(0)

print(embeddings.shape)

{'id': 'HAB_001_001', 'verse_id': 'HAB 1:1', 'audio': {'path': 'HAB_001_001.wav', 'array': array([-0.00360107, -0.00469971, -0.00457764, ..., -0.0045166 ,
       -0.00448608, -0.00445557]), 'sampling_rate': 44100}, 'verse_text': 'Neno alilopokea nabii Habakuki.', 'transcript': 'neno alilopokea nabii habakuki'}
{'id': 'HAB_001_001', 'verse_id': 'HAB 1:1', 'audio': {'path': 'HAB_001_001.wav', 'array': array([-0.0031652 , -0.00281096,  0.00320882, ..., -0.00424137,
       -0.00521683,  0.        ]), 'sampling_rate': 16000}, 'verse_text': 'Neno alilopokea nabii Habakuki.', 'transcript': 'neno alilopokea nabii habakuki'}
torch.Size([1, 512])


In [ ]:
# from datasets import load_dataset

# embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

# import torch

# speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# print(speaker_embeddings.shape)

In [ ]:
inputs = processor(text=text, return_tensors="pt")

In [ ]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [ ]:
speech = model.generate_speech(inputs["input_ids"], embeddings, vocoder=vocoder)


In [ ]:
print(text)

mbolea ya kuku hudza na vidudu here huke wa kuku


In [ ]:
from IPython.display import Audio

Audio(speech, rate=16000)


#Automatic speech recognition (asr nyf bible)


##How to fine-tune an ASR system with the traininer API

In [ ]:
dataset_name = "kazeric/Giriama_bible_dataset_no_split"
dataset_config_name = "sw"
model_name = "openai/whisper-small"
model_revision = "main"
dataset_text_column = "verse_text"
dataset_audio_column = "file_path"
output_dir = "whisper_small_nyf_bible"
hub_id = "kazeric/whisper_small_nyf_bible"
task="transcribe"
language = "swahili"
max_steps=1000
eval_steps=500
save_steps=500
revision="main"
commit_message="first commit"

In [ ]:
# !pip install --upgrade torch torchvision torchaudio

I might not spend time here as this is the back up plan, but if speech t5 doesnt work out then we can use the whisper model to

- train ASR in swahili then giriama to utilise transfere learning
- train ASR in giriama alone to check it viability as well

In [ ]:
!pip install datasets

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# create the branch to be used
from huggingface_hub import create_branch

if revision == "":
    revision = "main"
else:
  try:
    create_branch(repo_id=hub_id, branch=revision)
  except:
    print("Branch already exists")

Branch already exists


In [ ]:
from datasets import load_dataset, Audio
import random


dataset = load_dataset(dataset_name, split="train")

dataset = dataset.shuffle(seed=42)

random.seed(42)

dataset = dataset.train_test_split(test_size=0.2)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['verse_text', 'file_path'],
        num_rows: 5740
    })
    test: Dataset({
        features: ['verse_text', 'file_path'],
        num_rows: 1435
    })
})


In [ ]:
dataset = dataset.select_columns([dataset_audio_column, dataset_text_column])


In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    model_name, language=language, task=task
)

In [ ]:
dataset["train"][0]

{'file_path': {'path': '12.wav',
  'array': array([-0.01145935, -0.01039124, -0.00746155, ..., -0.0124054 ,
         -0.01472473, -0.0186615 ]),
  'sampling_rate': 22050},
 'verse_text': "K'uzhona zho Anania akidzigidzya akiamba, “Bwana, mut'u mimi nidzasikira manji sana dzulu za mut'u yuyu, viryahu arizhoahendera nav̱isi o at'u a kwako ko Jerusalemu."}

In [ ]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
dataset = dataset.cast_column(dataset_audio_column, Audio(sampling_rate=sampling_rate))

In [ ]:
print(len(dataset["train"]))
print(len(dataset["test"]))

5740
1435


In [ ]:
def prepare_dataset(example):
    audio = example[dataset_audio_column]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example[dataset_text_column],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

[link text](https://)why are we removing the column names

In [ ]:
dataset = dataset.map(
    prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=1
)

Map:   0%|          | 0/5740 [00:00<?, ? examples/s]

Map:   0%|          | 0/1435 [00:00<?, ? examples/s]

In [ ]:
max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

In [ ]:
dataset["train"] = dataset["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)


Filter:   0%|          | 0/5740 [00:00<?, ? examples/s]

In [ ]:
dataset["train"]

Dataset({
    features: ['input_features', 'labels', 'input_length'],
    num_rows: 5717
})

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
!pip install evaluate

In [ ]:
!pip install jiwer

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
     model.generate, language=language, task=task, use_cache=True
)

what exactly are steps and what are they used for?

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=max_steps,
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    eval_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=save_steps,
    eval_steps=eval_steps,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    hub_model_id=hub_id,
)


In [ ]:
from transformers import Seq2SeqTrainer, EarlyStoppingCallback


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

<ipython-input-59-7503918aeb28>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


how and where is the number of epochs specified in the model?

what is this error and can it be solved for to bring better results

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Wer Ortho,Wer
500,0.172700,0.244998,22.891847,15.535445
1000,0.085200,0.215411,19.221298,12.223351


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=1000, training_loss=0.30637848043441773, metrics={'train_runtime': 6062.8013, 'train_samples_per_second': 2.639, 'train_steps_per_second': 0.165, 'total_flos': 4.61101752631296e+18, 'train_loss': 0.30637848043441773, 'epoch': 2.793296089385475})

In [ ]:
kwargs = {
    "dataset": dataset_name,  # a 'pretty' name for the training dataset
    "language": "nyf",
    "model_name": output_dir,  # a 'pretty' name for your model
    "finetuned_from": model_name,
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs,commit_message=commit_message , revision=revision)

CommitInfo(commit_url='https://huggingface.co/kazeric/whisper_small_nyf_bible/commit/d9368668bf2ca1e12a70ca04d1950f309c63fa31', commit_message='first commit', commit_description='', oid='d9368668bf2ca1e12a70ca04d1950f309c63fa31', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kazeric/whisper_small_nyf_bible', endpoint='https://huggingface.co', repo_type='model', repo_id='kazeric/whisper_small_nyf_bible'), pr_revision=None, pr_num=None)

#Finetuning speecht5 (t5 nyf bible)




Major variables

In [ ]:
dataset_name = "kazeric/Giriama_bible_dataset_no_split"
dataset_config_name = "sw"
model_name = "Lingua-Connect/speecht5_sw_bible"
model_revision = "main"
dataset_text_column = "verse_text"
dataset_audio_column = "file_path"
output_dir = "speecht5_sw_bible"
hub_id = "Lingua-Connect/speecht5_sw_bible"
max_steps=4000
eval_steps=80
save_steps=480
revision="wandb"
commit_message="wandb test commit"


In [ ]:
!pip install transformers datasets soundfile speechbrain accelerate

In [ ]:
!pip install --upgrade datasets huggingface_hub fsspec

  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# create the branch to be used
from huggingface_hub import create_branch

if revision == "":
    revision = "main"
else:
  try:
    create_branch(repo_id=hub_id, branch=revision)
  except:
    print("Branch already exists")

Branch already exists


In [ ]:
from datasets import load_dataset, Audio
import random


dataset = load_dataset(dataset_name, split="train")

dataset = dataset.shuffle(seed=42)

random.seed(42)

dataset = dataset.train_test_split(test_size=0.2)


In [ ]:
dataset["train"] = dataset["train"].cast_column(dataset_audio_column, Audio(sampling_rate=16000))
dataset["test"]= dataset["test"].cast_column(dataset_audio_column, Audio(sampling_rate=16000))

In [ ]:
from transformers import SpeechT5Processor

processor = SpeechT5Processor.from_pretrained(model_name, revision=model_revision)

special_tokens_map.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

In [ ]:
tokenizer = processor.tokenizer

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch[dataset_text_column])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs_train = dataset["train"].map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset["train"].column_names,
)


vocabs_test = dataset["test"].map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset["test"].column_names,
)

tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

dataset_train_vocab = set(vocabs_train["vocab"][0])
dataset_test_vocab = set(vocabs_test["vocab"][0])

print(f"The unique train vocabs{dataset_train_vocab - tokenizer_vocab}")
print(f"The unique test vocabs{dataset_test_vocab - tokenizer_vocab}")

Map:   0%|          | 0/5740 [00:00<?, ? examples/s]

Map:   0%|          | 0/1435 [00:00<?, ? examples/s]

The unique train vocabs{'1', ' ', '”', '2', '‘', '̱', '6', '“', '’', '瘫', '0'}
The unique test vocabs{'1', ' ', '”', '‘', '̱', '�', '4', '“', '’', '瘫', '0'}


In [ ]:

replacements = [
    ('0', ""),
    (' ', ""),
    ('1', ""),
    ("2", ""),
    ('‘', ""),
    ('”', ""),
    ('6', ""),
    ('̱', ""),
    ('4', ""),
    ('瘫', ""),
    ('’', ""),
    ('“', ""),
    ('�', "")
]


def cleanup_text(inputs):
    for src, dst in replacements:
        inputs[dataset_text_column] = inputs[dataset_text_column].replace(src, dst)
    return inputs


dataset["train"] = dataset["train"].map(cleanup_text)
dataset["test"] = dataset["test"].map(cleanup_text)

Map:   0%|          | 0/5740 [00:00<?, ? examples/s]

Map:   0%|          | 0/1435 [00:00<?, ? examples/s]

In [ ]:
# from collections import defaultdict

# speaker_counts = defaultdict(int)

# for example in dataset["train"]:
#     speaker_counts[example["speaker_id"]] += 1

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure()
# plt.hist(speaker_counts.values(), bins=20)
# plt.ylabel("Speakers")
# plt.xlabel("Examples")
# plt.show()

In [ ]:
# def select_speaker(speaker_id):
#     return 100 <= speaker_counts[speaker_id] <= 400


# dataset["train"] = dataset["train"].filter(select_speaker, input_columns=["speaker_id"])
# dataset["test"] = dataset["test"].filter(select_speaker, input_columns=["speaker_id"])



In [ ]:
# len(set(dataset["speaker_id"]))

In [ ]:
len(dataset)

2

In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/tmp/speechbrain/spkrec-xvect-voxceleb/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in /tmp/speechbrain/spkrec-xvect-voxceleb.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Using symlink found at '/tmp/speechbrain/spkrec-xvect-voxceleb/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /tmp/speechbrain/spkrec-xvect-voxceleb/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Using symlink found at '/tmp/speechbrain/spkrec-xvect-voxceleb/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /tmp/speechbrain/spkrec-xvect-voxceleb/mean_var_norm_emb.ck

In [ ]:
def prepare_dataset(example):
    audio = example[dataset_audio_column]

    example = processor(
        text=example[dataset_text_column],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [ ]:
# processed_example = prepare_dataset(dataset[0])
# list(processed_example.keys())

In [ ]:
# processed_example["speaker_embeddings"].shape

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure()
# plt.imshow(processed_example["labels"].T)
# plt.show()

In [ ]:
dataset["train"] = dataset["train"].map(prepare_dataset, remove_columns=dataset["train"].column_names)
dataset["test"] = dataset["test"].map(prepare_dataset, remove_columns=dataset["test"].column_names)

Map:   0%|          | 0/5740 [00:00<?, ? examples/s]

Map:   0%|          | 0/1435 [00:00<?, ? examples/s]

In [ ]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200


dataset["train"] = dataset["train"].filter(is_not_too_long, input_columns=["input_ids"])
dataset["test"] = dataset["test"].filter(is_not_too_long, input_columns=["input_ids"])


Filter:   0%|          | 0/5740 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1435 [00:00<?, ? examples/s]

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [ ]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(model_name, revision=model_revision)

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=max_steps,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=save_steps,
    eval_steps=eval_steps,
    logging_steps=25,
    report_to="wandb",
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
    hub_model_id=hub_id,
)

In [ ]:
from transformers import Seq2SeqTrainer, EarlyStoppingCallback

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

<ipython-input-53-dd4173063256>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kahindie873 (kahindie873-jomo-kenyatta-university-of-agriculture-and-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
80,0.468000,0.434533
160,0.462600,0.435642
240,0.462000,0.434691
320,0.457500,0.434371
400,0.480800,0.434734
480,0.462400,0.434694
560,0.467200,0.434635


TrainOutput(global_step=560, training_loss=0.46679775970322746, metrics={'train_runtime': 1411.0971, 'train_samples_per_second': 45.355, 'train_steps_per_second': 2.835, 'total_flos': 1398356359420680.0, 'train_loss': 0.46679775970322746, 'epoch': 1.6816816816816815})

In [ ]:
kwargs = {
    "dataset": dataset_name,  # a 'pretty' name for the training dataset
    "language": "nyf",
    "model_name": output_dir,  # a 'pretty' name for your model
    "finetuned_from": model_name,
    "tasks": "text-to-speech",
}

In [ ]:
trainer.push_to_hub(**kwargs,commit_message=commit_message , revision=revision)

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Lingua-Connect/speecht5_sw_bible/commit/88ec2133269a61c61c0366507e808c225fb1244f', commit_message='wandb test commit', commit_description='', oid='88ec2133269a61c61c0366507e808c225fb1244f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Lingua-Connect/speecht5_sw_bible', endpoint='https://huggingface.co', repo_type='model', repo_id='Lingua-Connect/speecht5_sw_bible'), pr_revision=None, pr_num=None)

#Diagnostics for errors

##hugging_face hub and datasets error

In [ ]:
from datasets import load_dataset
import sys
import os

# Print Python version and environment information
print(f"Python version: {sys.version}")
print(f"Current working directory: {os.getcwd()}")

# Explicitly define the dataset name
dataset_name = "kazeric/Giriama_bible_dataset_no_split"
print(f"Attempting to load dataset: {dataset_name}")

# Try with various parameters
try:
    print("Attempt 1: Basic loading")
    dataset = load_dataset(dataset_name)
    print("Success!")
except Exception as e:
    print(f"Error in Attempt 1: {type(e).__name__}: {e}")

try:
    print("\nAttempt 2: Loading with download_mode='force_redownload'")
    from datasets import DownloadMode
    dataset = load_dataset(dataset_name, download_mode=DownloadMode.FORCE_REDOWNLOAD)
    print("Success!")
except Exception as e:
    print(f"Error in Attempt 2: {type(e).__name__}: {e}")

try:
    print("\nAttempt 3: Loading a known public dataset")
    test_dataset = load_dataset("mnist", split="train")
    print("Success loading MNIST!")
except Exception as e:
    print(f"Error loading MNIST: {type(e).__name__}: {e}")

# Print version information for relevant libraries
try:
    import datasets
    print(f"\nDatasets library version: {datasets.__version__}")
except Exception as e:
    print(f"Error getting datasets version: {e}")

try:
    import huggingface_hub
    print(f"Huggingface Hub version: {huggingface_hub.__version__}")
except Exception as e:
    print(f"Error getting huggingface_hub version: {e}")

In [ ]:
#solution

!pip install --upgrade datasets huggingface_hub fsspec